1. 通过python 执行 gsql 文件；

In [1]:
import subprocess

In [ ]:
# 所有点边拿到， 然后跑pagerank

In [ ]:
print(1)

In [1]:
!pwd

/home/tigergraph/GraphProject/OneMonthGraph/Querys


In [ ]:
Use graph workNet                                                                                                                                                                                       

# sub query
CREATE OR REPLACE QUERY sub_query() FOR GRAPH workNet                                                                                                    
RETURNS(SetAccum<VERTEx>){
SETAccum @@unique_nodes;                                                                                                                                                                               
select s from init:s -(worksFor:e)->:t                                                                                                                                                                   
ACCUM @@unique_nodes+=t;                                                                                                                                                                                
POST-ACCUm @@unique_nodes+=s;                                                                                                                                                                           
}


# main query

CREATE QUERY pageRank_attr (FLOAT maxChange, INT maxIter, FLOAT damping) FOR GRAPH workNet
{
    MaxAccum<float> @@maxDiff = 9999; # max score change in an iteration
    SumAccum<float> @received_score = 0; # sum of scores each vertex receives FROM neighbors
    SumAccum<float> @score = 1;   # Initial score for every vertex is 1.

    Start = sub_query();   #  Start with all vertices of specified type(s)
    WHILE @@maxDiff > maxChange LIMIT maxIter DO
            @@maxDiff = 0;
            V = SELECT s
                FROM Start:s -(Friend:e)-> :t
                ACCUM t.@received_score += s.@score/(s.outdegree("Friend")) 
                POST-ACCUM s.@score = (1.0-damping) + damping * s.@received_score,
                           s.@received_score = 0,
                           @@maxDiff += abs(s.@score - s.@score');
    END; # END WHILE loop
    V = SELECT s FROM Start:s
    POST-ACCUM s.score = s.@score;
}
                                     
INSTALL QUERY pageRank_attr                                                                                                                                                                                   
RUN QUERY pageRank_attr(10, 4, 0.6)                    
                                            

In [8]:
a = '''person1,company1,2016,1,1
person1,company2,2014,3,0
person2,company1,2015,7,1
person2,company2,2012,6,0
person3,company1,2016,6,1
person4,company2,2013,2,1
person5,company2,2016,4,0
person6,company1,2015,1,1
person7,company2,2016,3,0
person7,company3,2014,1,0
person8,company1,2013,2,1
person9,company2,2015,12,1
person9,company3,2016,11,1
person10,company1,2016,2,1
person10,company3,2014,5,0
person11,company5,2016,5,1
person12,company4,2014,1,1'''

In [14]:
res = [i.split(",")[:2] for i in a.split("\n")]

In [13]:
import networkx as nx

In [15]:
>>> G = nx.Graph()   # or DiGraph, MultiGraph, MultiDiGraph, etc
>>> G.add_edges_from(res) # using a list of edge tuples

In [17]:
>>> pr = nx.pagerank(G, alpha=0.6)

In [27]:
sorted(pr.items(), key=lambda kv: kv[1])

[('person4', 0.036209774589269564),
 ('person5', 0.036209774589269564),
 ('person3', 0.03706268219671062),
 ('person6', 0.03706268219671062),
 ('person8', 0.03706268219671062),
 ('person1', 0.0497430450212743),
 ('person2', 0.0497430450212743),
 ('person7', 0.04995965612902013),
 ('person9', 0.04995965612902013),
 ('person10', 0.05081256373646119),
 ('person11', 0.058823529411764705),
 ('company5', 0.058823529411764705),
 ('person12', 0.058823529411764705),
 ('company4', 0.058823529411764705),
 ('company3', 0.06874832448720235),
 ('company2', 0.1268014618050533),
 ('company1', 0.13533053425496439)]

In [ ]:
# 根据两个cutoff 筛选所有点；

In [ ]:
Use graph workNet                                                                                                                                                                                       
CREATE OR REPLACE QUERY qb_test(vertex<person> ps, uint cut1, uint cut2) FOR GRAPH workNet{                                                                                                             
SetAccum<DATETIME> @@group;                                                                                                                                                                             
OrAccum @@inCutoff;                                                                                                                                                                                     
    
init = {ps};                                                                                                                                                                                            
res = select s from init:s                                                                                                                                                                              
ACCUM                                                                                                                                                                                                   
@@group += s.skillSet;                                                                                                                                                                                  
FOREACH g IN @@group DO                                                                                                                                                                                 
if g between cut1 and cut2 then @@inCutoff += True; END;END;                                                                                                                                            
print @@group;                                                                                                                                                                                          
print @@inCutoff;                                                                                                                                                                                       
}                                                                                                                                                                                                       
INSTALL QUERY qb_test                                                                                                                                                                                   
RUN QUERY qb_test("person1",4,8) 